# Assignment 1
CIFAR-10

In [1]:
# import some necessary packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as tv_datasets
import torchvision.transforms as tv_transforms

from tqdm import tqdm

In [2]:
# some experimental setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = 128
batch_size = 256
num_workers = 8
# print_every = 200

optim_name = "AdamW"
optim_kwargs = dict(
    lr=3e-4,
)

# preprocessing pipeline for input images
transformation = dict()
for data_type in ("train", "test"):
    is_train = data_type=="train"
    transformation[data_type] = tv_transforms.Compose(([
        tv_transforms.RandomRotation(degrees=15),
        tv_transforms.RandomHorizontalFlip(),
        tv_transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0.1),
        tv_transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    ] if is_train else []) +
    [
        tv_transforms.ToTensor(),
        tv_transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

print(f"device: {device}")


device: cuda:0


In [3]:
# prepare datasets
dataset, loader = {}, {}
for data_type in ("train", "test"):
    is_train = data_type=="train"
    dataset[data_type] = tv_datasets.CIFAR10(
        root="./data", train=is_train, download=True, transform=transformation[data_type],
    )
    loader[data_type] = torch.utils.data.DataLoader(
        dataset[data_type], batch_size=batch_size, shuffle=is_train, num_workers=num_workers,
    )

print(f"train sample: {len(dataset['train'])}, test samples: {len(dataset['test'])}")
print(f"train batches: {len(loader['train'])}, test batches: {len(loader['test'])}")

Files already downloaded and verified
Files already downloaded and verified
train sample: 50000, test samples: 10000
train batches: 196, test batches: 40


### Baseline

In [4]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 128, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2), nn.Dropout(0.3),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2), nn.Dropout(0.3),
            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2), nn.Dropout(0.3),
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.Linear(512, 256), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.Linear(256, 128), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        return self.model(x)


### Residual Mechanism

In [5]:
class MyResBlock(nn.Module):
    def __init__(self, in_c, out_c, stride=1):
        super(MyResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_c, out_c, 3, stride=stride, padding=1)
        self.conv2 = nn.Conv2d(out_c, out_c, 3, padding=1)
        if in_c != out_c:
            self.conv3 = nn.Conv2d(in_c, out_c, 1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_c)
        self.bn2 = nn.BatchNorm2d(out_c)

    def forward(self, x):
        y = F.relu(self.bn1(self.conv1(x)))
        y = F.relu(self.bn2(self.conv2(y)))
        if self.conv3 is not None:
            x = self.conv3(x)
        return F.relu(x + y)

class MyResnet(nn.Module):
    def __init__(self, num_block=1):
        super(MyResnet, self).__init__()
        self.before_res = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),
        )
        self.res = nn.Sequential(
            *[MyResBlock(64, 64) for _ in range((num_block - 1) * 1)],
            MyResBlock(64, 128, 2), *[MyResBlock(128, 128) for _ in range((num_block - 1) * 2)], nn.Dropout(0.05),
            MyResBlock(128, 256, 2), *[MyResBlock(256, 256) for _ in range((num_block - 1) * 4)], nn.Dropout(0.05),
            MyResBlock(256, 512, 2), *[MyResBlock(512, 512) for _ in range((num_block - 1) * 2)], nn.Dropout(0.05),
            MyResBlock(512, 1024, 2), *[MyResBlock(1024, 1024) for _ in range((num_block - 1) * 1)], nn.Dropout(0.05),
        )
        self.after_res = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(), nn.Linear(1024, 10)
        )

    def forward(self, x):
        return self.after_res(self.res(self.before_res(x)))


### Attention

In [6]:
# our network architecture
class MyAttentionBlock(nn.Module):
    def __init__(self, d_attn, num_heads=4):
        super(MyAttentionBlock, self).__init__()
        self.num_heads = num_heads
        self.mha = nn.MultiheadAttention(embed_dim=d_attn, num_heads=num_heads, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Conv2d(d_attn, d_attn*4, 1), nn.ReLU(inplace=True),
            nn.Conv2d(d_attn*4, d_attn, 1),
        )
        self.bn1 = nn.BatchNorm2d(d_attn)
        self.bn2 = nn.BatchNorm2d(d_attn)

    def forward(self, x):
        B, C, W, H = x.shape
        x_flat = x.flatten(2).permute(0, 2, 1)  # B, L, C
        attn_out, _ = self.mha(x_flat, x_flat, x_flat)
        attn_out = attn_out.permute(0, 2, 1).reshape(B, C, W, H)
        attn_out = F.relu(attn_out + x)
        attn_out = self.bn1(attn_out)
        out = self.ffn(attn_out) + attn_out
        out = self.bn2(out)
        return out

class MyAttention(nn.Module):
    def __init__(self, num_block=1):
        super(MyAttention, self).__init__()
        self.before_attn = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),
        )
        self.attn = nn.Sequential(
            MyResBlock(64, 128, 2), *[MyAttentionBlock(128) for _ in range(num_block)],
            MyResBlock(128, 256, 2), *[MyAttentionBlock(256) for _ in range(num_block)],
            MyResBlock(256, 512, 2), *[MyAttentionBlock(512) for _ in range(num_block)],
        )
        self.after_attn = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(), nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.after_attn(self.attn(self.before_attn(x)))


In [7]:
# move to device
model = MyResnet(num_block=2)
model.to(device)

# print the number of parameters
print(f"number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad) / 1_000_000:.2f}M")

number of parameters: 53.24M


## Start Training

In [8]:
# the network optimizer
optimizer = getattr(optim, optim_name)(model.parameters(), **optim_kwargs)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# loss function
criterion = nn.CrossEntropyLoss()

# training loop
model.train()
for epoch in range(num_epochs):

    running_loss = 0.0
    for i, (img, target) in tqdm(enumerate(loader["train"]), total=len(loader["train"])):
        img, target = img.to(device), target.to(device)

        pred = model(img)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % print_every == print_every - 1:
        #     print(f"[epoch={epoch + 1:3d}, iter={i + 1:5d}] loss: {running_loss / print_every:.3f}")
        #     running_loss = 0.0
    print(f"[epoch={epoch + 1:3d}] loss: {running_loss / len(loader['train']):.3f}")
    scheduler.step()

print("Finished Training")


100%|██████████| 196/196 [00:10<00:00, 18.08it/s]

[epoch=  1] loss: 1.567



100%|██████████| 196/196 [00:10<00:00, 18.91it/s]

[epoch=  2] loss: 1.122



100%|██████████| 196/196 [00:10<00:00, 19.21it/s]

...

[epoch=126] loss: 0.003



100%|██████████| 196/196 [00:10<00:00, 19.14it/s]

[epoch=127] loss: 0.004



100%|██████████| 196/196 [00:10<00:00, 19.04it/s]

[epoch=128] loss: 0.004
Finished Training


## Evaluating its accuracy

In [9]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for img, target in loader["test"]:
        img, target = img.to(device), target.to(device)

        # make prediction
        pred = model(img)

        # accumulate
        total += len(target)
        correct += (torch.argmax(pred, dim=1) == target).sum().item()

print(f"Accuracy of the network on the {total} test images: {100 * correct / total:.2f}%")

Accuracy of the network on the 10000 test images: 93.24%
